# Profile analysis - ALL profiles at once

This notebooks takes output from ```ADV_data_01_process_vXX.ipynb``` and performs analysis on all profile data 

Needs:
- a file called ```summary_velocity_mean.csv``` which is the natural output of the above referenced notebook
- a file called ```summary_coniditions_locations.csv``` to provide spatial information for the 

User input:
- Data folder
- Profile $x$ location
- Water temperature

Version/Change log:
- V01: works with the big master dataset. Some changes in the column names were made from V01 to work with the output form from master Google sheet

## Loads

In [1]:
# %pylab inline
import numpy as np
from numpy import log, sin, cos, exp, tan, pi
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

%matplotlib inline
%config InlineBackend.figure_format='retina' # hig-res plots for a Retina display, uncomment while working

# uncomment the two lines below when exporting to a pdf
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png','pdf')

## Functions

In [2]:
#returns the clear-water density in kg/m^3 as a function of temperature [in deg C] and salinity [ppt]
def rho_cw(T,S): 
    rho_fresh=1000*(1-(T +288.9414)/(508929.2*(T+68.12963))*(T-3.9863)**2)
    Acoef = 0.824493 - 0.0040899*T + 0.000076438*T**2 -0.00000082467*T**3 + 0.0000000053675*T**4
    Bcoef = -0.005724 + 0.00010227*T - 0.0000016546*T**2
    return rho_fresh + Acoef*S + Bcoef*S**(3/2) + 0.00048314*S**2 

# returns the clear water kinimatic viscosity in [m^2/s] as a function of temperature [in deg C].
def visc(T):
    return 2.7488e-07+1.4907e-06*exp(-0.034812*T)

def swlogfit(z, ustar):
    Bs = 5.5
    return (ustar/kappa)*log(ustar*z/nu)+ustar*Bs

def rwlogfit(z, ks, ustar):
    Br = 8.5
    z0 = 0.25*ks
    # z0 = 0
    return (ustar/kappa)*log((z+z0)/ks)+ustar*Br

def rwlogfit2(z, z0, ks, ustar):
    Br = 8.5
    # z0 = 0.25*ks
    # z0 = 0
    return (ustar/kappa)*log((z+z0)/ks)+ustar*Br

## User input

In [7]:
datadir = 'test_data/' # path to the data
profilex = 5.12

T = 10 # water temperature in deg C

w = 1.2
B = 8.5
kappa = 0.41

## Load the data and condition

In [8]:
output_path=datadir+'/summary_profile_anal_all.csv'

# data input files
veldatafile = 'summary_velocity_mean.csv'
conditionlocationfile = 'summary_coniditions_locations.csv'

fig_folder = 'Figures'
# Make the output folder
if os.path.isdir(fig_folder) != 1:
    os.mkdir(fig_folder)

data = pd.read_csv(datadir+veldatafile) # reads in the data
info = pd.read_csv(datadir+conditionlocationfile) # reads in the data

info.insert(loc=1,column='Run',value=info.File.str[0:5])
info.insert(loc=2,column='Point',value=info.File.str[6:8].astype(int))
info.insert(loc=4,column='S',value=info.S_p/100)

runs = info.Run.unique()

## Define the run and profile to analyze

Uses the user input of the profile location and run to pull out data from the two ```.csv``` files and create a new ```profile``` dataframe to work with

In [9]:
for j in range(0,len(runs)):
    run = runs[j]
    
    print('Analyzing profile =',run)

    # build the profile to analyze -----------------------------------
    
    pinfo = info[(info.Run == run)&(info.x_m == profilex)].copy() # pull out the profile for the run
    pinfo = pinfo.sort_values(by=['Point']).reset_index(drop=True) # sort the data on the points in case something is out of order

    first = pinfo.Point.min() # define which points are part of the profile - the first
    last = pinfo.Point.max()  # define which points are part of the profile - the last
    pdata = data[(data.Run == run)&(data.Point >= first)&(data.Point <= last)].copy() # pull out the profile for the run
    pdata = pdata.sort_values(by=['Point']).reset_index(drop=True)   # sort the data on the points in case something is out of order

    # drop data in the combine... drop run and point since they are duplicate; snr and cor dropped for visual check... might want them if looking at data quality
    pinfo = pinfo.drop(columns=['Run','Point'])
    pdata = pdata.drop(columns=['snr1','snr2','snr3','snr4','cor1','cor2','cor3','cor4'])

    profile = pd.concat([pinfo,pdata], axis=1)
    
    # resort the dataframe for better plotting and curve fitting

    profile = profile.sort_values(by=['z_cm']).reset_index(drop=True) # sort the data on the points in case something is out of order

    # prelim calcs -----------------------------------

    g = 9.81
    nu = visc(T)
    rho = rho_cw(T,0)
    S = profile.S[0]
    h = profile.h_cm[0]/100
    R = h*w/(w+2*h)

    taub_ds = rho*g*R*S # bed shear stress from depth slope product
    ustar_ds = np.sqrt(taub_ds/rho)
    
    # profile analysis -----------------------------------
    
    # create 1D arrays for ease
    u=np.array(profile.uavg_m_s.copy())
    z=np.array(profile.z_cm.copy()/100)

    # depth avg vel
    Uavg = np.trapz(u,z)/(z[-1]-z[0]) # depth average u for the profile
    QUavg = Uavg*h*w
    f_ds = 8*taub_ds/(rho*Uavg**2)

    # profile analysis

    X = (1/kappa)*log(z) # create loged velocity variable
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,u)

    ustar = slope
    ks = exp(kappa*(B-(intercept/ustar)))
    u_calc=ustar*((1/kappa)*log(z/ks)+B)
    f = 8*ustar**2/Uavg**2
   
    slope_uw, taub_uw, r_value, p_value, std_err = stats.linregress(profile.z_cm,-rho*profile.uw_bar)

    ustar_uw = np.sqrt(taub_uw/rho)
    f_uw = 8*ustar_uw**2/Uavg**2
    
    ptable = profile.loc[0:0][['Run','Q_cms','S','T_cm','x_m','h_cm']].copy()
    ptable['uavg_m_s']= Uavg
    ptable['Q_Uavg_cms']=QUavg
    ptable['ks_m']=ks
    ptable['ustar_profile_m_s']=ustar
    ptable['ustar_ds_m_s']= ustar_ds
    ptable['ustar_uw_m_s'] = ustar_uw
    ptable['f_ds'] = f_ds
    ptable['f_profile'] = f
    ptable['f_uw'] = f_uw
    ptable['taub_profile_Pa']= (f/8)*rho*Uavg**2
    ptable['taub_ds_Pa']= taub_ds
    ptable['taub_uw_Pa'] = (f_uw/8)*rho*Uavg**2
    
    if j == 0:
        summary=ptable.copy()
    else:
        # summary = summary.append(ptable, ignore_index = True)
        summary = pd.concat((summary,ptable),axis=0,ignore_index=True)

#     ptable.to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path))

display(summary)
summary.to_csv(output_path, index=False)

Analyzing profile = Run1A
Analyzing profile = Run1B
Analyzing profile = Run1C
Analyzing profile = Run1D
Analyzing profile = Run2A
Analyzing profile = Run2B
Analyzing profile = Run2C
Analyzing profile = Run2D
Analyzing profile = Run3A
Analyzing profile = Run3B
Analyzing profile = Run3C
Analyzing profile = Run3D


,Run,Q_cms,S,T_cm,x_m,h_cm,uavg_m_s,Q_Uavg_cms,ks_m,ustar_profile_m_s,ustar_ds_m_s,ustar_uw_m_s,f_ds,f_profile,f_uw,taub_profile_Pa,taub_ds_Pa,taub_uw_Pa
0,NaN,0.099,0.0050,7.8,5.12,12.5,NaN,NaN,NaN,NaN,0.071233,NaN,NaN,NaN,NaN,NaN,5.072758,NaN
1,Run1B,0.113,0.0050,7.8,5.12,12.8,0.71394,0.109661,0.038353,0.076286,0.071934,0.065963,0.081215,0.091338,0.068291,5.817932,5.173099,4.349887
2,NaN,0.126,0.0050,7.8,5.12,14.5,NaN,NaN,NaN,NaN,0.075683,NaN,NaN,NaN,NaN,NaN,5.726429,NaN
3,NaN,0.139,0.0050,7.8,5.12,15.4,NaN,NaN,NaN,NaN,0.077530,NaN,NaN,NaN,NaN,NaN,6.009268,NaN
4,NaN,0.100,0.0075,8.0,5.12,11.7,NaN,NaN,NaN,NaN,0.084874,NaN,NaN,NaN,NaN,NaN,7.201619,NaN
5,NaN,0.113,0.0075,8.0,5.12,12.2,NaN,NaN,NaN,NaN,0.086368,NaN,NaN,NaN,NaN,NaN,7.457376,NaN
6,NaN,0.127,0.0075,8.0,5.12,13.3,NaN,NaN,NaN,NaN,0.089498,NaN,NaN,NaN,NaN,NaN,8.007761,NaN
7,NaN,0.139,0.0075,8.0,5.12,14.9,NaN,NaN,NaN,NaN,0.093712,NaN,NaN,NaN,NaN,NaN,8.779461,NaN
8,NaN,0.099,0.0075,8.0,5.12,10.4,NaN,NaN,NaN,NaN,0.080755,NaN,NaN,NaN,NaN,NaN,6.519647,NaN
9,NaN,0.112,0.0075,8.0,5.12,12.0,NaN,NaN,NaN,NaN,0.085776,NaN,NaN,NaN,NaN,NaN,7.355500,NaN
